In [8]:
# imports
import pandas as pd
import numpy as np
#import seborn
from sklearn.preprocessing import MinMaxScaler, KBinsDiscretizer


In [9]:
# import data
data = pd.read_csv("accepted_2007_to_2018Q4.csv", engine='c')
data = data.drop(['id', 'member_id', 'settlement_term','settlement_percentage', 'settlement_amount', 'settlement_date','settlement_status', 'debt_settlement_flag_date', 'hardship_last_payment_amount', 'hardship_payoff_balance_amount', 'orig_projected_additional_accrued_interest',
           'hardship_loan_status', 'hardship_dpd', 'hardship_length','payment_plan_start_date','hardship_end_date', 'hardship_start_date', 'hardship_amount', 'deferral_term', 'hardship_status', 'hardship_reason', 'hardship_type',
           'sec_app_mths_since_last_major_derog', 'sec_app_collections_12_mths_ex_med', 'sec_app_chargeoff_within_12_mths', 'sec_app_num_rev_accts', 'sec_app_open_act_il', 'sec_app_revol_util', 'sec_app_open_acc', 'sec_app_mort_acc',
           'sec_app_inq_last_6mths', 'sec_app_earliest_cr_line', 'sec_app_fico_range_high', 'sec_app_fico_range_low', 'verification_status_joint', 'dti_joint', 'annual_inc_joint', 'desc', 'url', 'revol_bal_joint','mths_since_last_record', 'mths_since_recent_bc_dlq', 'mths_since_last_major_derog', 'mths_since_recent_revol_delinq', 'next_pymnt_d',
           'il_util', 'mths_since_rcnt_il','mths_since_last_delinq', 'zip_code' ], axis=1)

#data = data.dropna()
#data
# maybe takeout these features as well:
# debt_settlement_flag

# I have left 'hardship_flag' included

# features removed on the second round of cleaning after looking at amount of missing rows in excel
#'mths_since_last_record', 'mths_since_recent_bc_dlq', 'mths_since_last_major_derog', 'mths_since_recent_revol_delinq', 'next_pymnt_d', 'mths_since_last_delinq'

# i removed zip codes since i couldnt categorize or group areas toghether, since the zip codes were obscured/sensored / hidden the original value

# idk what 'il_util' is but i removed it
data.convert_dtypes()
data.dtypes
display(data)

C:\Users\zachs\AppData\Local\Temp\ipykernel_21736\3345206350.py:2: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("accepted_2007_to_2018Q4.csv", engine='c')


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,...,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,N,Cash,N
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,...,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,N,Cash,N
2,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,...,50.0,0.0,0.0,218418.0,18696.0,6200.0,14877.0,N,Cash,N
3,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,Information Systems Officer,10+ years,...,0.0,0.0,0.0,381215.0,52226.0,62500.0,18000.0,N,Cash,N
4,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,...,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,N,Cash,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260696,40000.0,40000.0,40000.0,60 months,10.49,859.56,B,B3,Vice President,9 years,...,50.0,0.0,0.0,55970.0,28398.0,12300.0,42670.0,N,Cash,N
2260697,24000.0,24000.0,24000.0,60 months,14.49,564.56,C,C4,Program Manager,6 years,...,40.0,1.0,0.0,84664.0,62426.0,20700.0,58764.0,N,Cash,Y
2260698,14000.0,14000.0,14000.0,60 months,14.49,329.33,C,C4,Customer Service Technician,10+ years,...,50.0,0.0,0.0,163804.0,44215.0,9500.0,34169.0,N,Cash,N
2260699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Analysis on missing data:
# sort columns by amount of missing data in them

# Count the number of missing values in each column
missing_counts = data.isnull().sum().sort_values(ascending=False)


display(missing_counts)

# Sort columns by the number of missing values in descending order
sorted_columns = missing_counts.sort_values(ascending=False).index

# Reorder the DataFrame columns based on the sorted order
sorted_df = data[sorted_columns]

# sorted_df now contains the DataFrame with columns sorted by missing values
display(sorted_df)

all_util                866381
open_acc_6m             866163
inq_last_12m            866163
total_cu_tl             866163
open_il_24m             866162
                         ...  
last_pymnt_amnt             33
last_fico_range_high        33
last_fico_range_low         33
policy_code                 33
debt_settlement_flag        33
Length: 102, dtype: int64

,all_util,total_cu_tl,open_acc_6m,inq_last_12m,open_il_24m,inq_fi,max_bal_bc,open_rv_24m,open_act_il,open_il_12m,...,term,int_rate,installment,grade,sub_grade,verification_status,issue_d,loan_status,pymnt_plan,debt_settlement_flag
0,34.0,1.0,2.0,4.0,1.0,3.0,722.0,3.0,2.0,0.0,...,36 months,13.99,123.03,C,C4,Not Verified,Dec-2015,Fully Paid,n,N
1,29.0,0.0,1.0,6.0,1.0,0.0,6472.0,3.0,1.0,0.0,...,36 months,11.99,820.28,C,C1,Not Verified,Dec-2015,Fully Paid,n,N
2,65.0,5.0,0.0,1.0,4.0,2.0,2081.0,2.0,1.0,0.0,...,60 months,10.78,432.66,B,B4,Not Verified,Dec-2015,Fully Paid,n,N
3,45.0,1.0,1.0,0.0,1.0,0.0,6987.0,1.0,1.0,0.0,...,60 months,14.85,829.90,C,C5,Source Verified,Dec-2015,Current,n,N
4,78.0,1.0,1.0,3.0,3.0,2.0,9702.0,7.0,3.0,0.0,...,60 months,22.45,289.91,F,F1,Source Verified,Dec-2015,Fully Paid,n,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260696,51.0,0.0,0.0,2.0,1.0,3.0,5141.0,0.0,2.0,0.0,...,60 months,10.49,859.56,B,B3,Verified,Oct-2016,Current,n,N
2260697,73.0,0.0,0.0,1.0,2.0,0.0,6172.0,5.0,2.0,0.0,...,60 months,14.49,564.56,C,C4,Not Verified,Oct-2016,Charged Off,n,Y
2260698,84.0,1.0,0.0,2.0,2.0,3.0,4093.0,1.0,1.0,1.0,...,60 months,14.49,329.33,C,C4,Verified,Oct-2016,Current,n,N
2260699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Analysis on missing data:
# sort rows by amount of missing data in them

# Count the number of missing values in each row
missing_counts_rows = data.isnull().sum(axis=1).sort_values(ascending=False)

# Sort rows by the number of missing values in descending order
sorted_df_rows = data.loc[missing_counts_rows.sort_values(ascending=False).index]
# sorted_df now contains the DataFrame with rows sorted by missing values
display(missing_counts_rows)
display(sorted_df_rows)

2260700    102
421095     102
1654416    102
1654415    102
1651665    102
          ... 
501844       0
501845       0
501846       0
501848       0
0            0
Length: 2260701, dtype: int64

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
2260700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
651665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
983169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
877717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
877716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027520,20000.0,20000.0,20000.0,36 months,8.59,632.19,A,A5,Customer Success Coordinator,< 1 year,...,28.6,0.0,0.0,66307.0,25788.0,39000.0,17307.0,N,Cash,N
2027523,8500.0,8500.0,8500.0,36 months,12.79,285.54,C,C1,Driver,10+ years,...,33.3,1.0,0.0,17800.0,10302.0,14300.0,0.0,N,Cash,N
2027524,3000.0,3000.0,3000.0,36 months,13.99,102.52,C,C3,Gas mechanic,10+ years,...,60.0,0.0,0.0,307403.0,120172.0,9100.0,105274.0,N,Cash,N
2027525,11200.0,11200.0,11200.0,36 months,10.99,366.63,B,B4,Project Manager,< 1 year,...,41.7,0.0,0.0,423733.0,122191.0,42700.0,91893.0,N,Cash,N


In [12]:
data_droppedna = data.dropna()
display(data_droppedna)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,...,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,N,Cash,N
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,...,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,N,Cash,N
4,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,...,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,N,Cash,N
6,20000.0,20000.0,20000.0,36 months,9.17,637.58,B,B2,Vice President of Recruiting Operations,10+ years,...,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,N,Cash,N
7,20000.0,20000.0,20000.0,36 months,8.49,631.26,B,B1,road driver,10+ years,...,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,N,Cash,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260691,32000.0,32000.0,32000.0,60 months,14.49,752.74,C,C4,Sales Manager,3 years,...,0.0,0.0,0.0,524379.0,122872.0,15800.0,23879.0,N,Cash,N
2260692,16000.0,16000.0,16000.0,60 months,12.79,362.34,C,C1,Manager,10+ years,...,50.0,3.0,0.0,87473.0,65797.0,10100.0,73473.0,N,Cash,N
2260693,24000.0,24000.0,24000.0,60 months,10.49,515.74,B,B3,Current Operations Officer,4 years,...,0.0,0.0,2.0,128902.0,31320.0,93700.0,28602.0,N,Cash,N
2260695,24000.0,24000.0,24000.0,60 months,10.49,515.74,B,B3,Database Administrator,10+ years,...,42.9,0.0,1.0,227883.0,140118.0,27900.0,172283.0,N,Cash,N


In [13]:

# apply normalization techniques 
#df_sklearn = data.copy() 
#for column  in df_sklearn.columns:
#    if df_sklearn[column].dtype == np.float64:
#        df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1)) 
#display(df_sklearn) 

In [29]:
#discretise Data

discretise_data = data_droppedna.copy() 

#for column  in discretise_data.columns:
#    if discretise_data[column].dtype == np.float64:
#        discretise_data[column] = pd.cut(discretise_data[column], bins= 10, labels= [0,1,2,3,4,5,6,7,8,9])
#display(discretise_data) 

discretise_data['loan_amnt'] = pd.cut(discretise_data['loan_amnt'], bins= 50)
discretise_data['funded_amnt'] = pd.cut(discretise_data['funded_amnt'], bins= 50)
discretise_data['funded_amnt_inv'] = pd.cut(discretise_data['funded_amnt_inv'], bins= 50)
discretise_data['int_rate'] = pd.cut(discretise_data['int_rate'], bins= 50)
discretise_data['installment'] = pd.cut(discretise_data['installment'], bins = 50)





est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
annual_inc_dataframe = pd.DataFrame(discretise_data['annual_inc'])
est.fit(annual_inc_dataframe)
annual_inc_dataframe = est.transform(annual_inc_dataframe)
discretise_data['annual_inc'] = annual_inc_dataframe

discretise_data['loan_status'] = discretise_data['loan_status'].apply(lambda x: 'Fully Paid' if 'Fully Paid' in x else x)
#discretise_data['loan_status'] = discretise_data['loan_status'].apply(lambda x: 'Late' if 'Late' in x else x)

# 'Current'
values_to_keep = ['Fully Paid','Current', 'Charged Off']
pattern = '|'.join(values_to_keep)

discretise_data = discretise_data[discretise_data['loan_status'].str.contains(pattern)]


est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
dti_dataframe = pd.DataFrame(discretise_data['dti'])
est.fit(dti_dataframe)
dti_dataframe = est.transform(dti_dataframe)
discretise_data['dti'] = dti_dataframe


est = KBinsDiscretizer(n_bins= 31, encode='ordinal', strategy='kmeans')
delinq_2yrs_dataframe = pd.DataFrame(discretise_data['delinq_2yrs'])
est.fit(delinq_2yrs_dataframe)
delinq_2yrs_dataframe = est.transform(delinq_2yrs_dataframe)
discretise_data['delinq_2yrs'] = delinq_2yrs_dataframe
#discretise_data['delinq_2yrs'] = pd.cut(discretise_data['delinq_2yrs'], bins= 3)


discretise_data['earliest_cr_line'] = discretise_data['earliest_cr_line'].apply(lambda x: x.split('-')[1])

discretise_data['fico_range_low'] = pd.cut(discretise_data['fico_range_low'], bins= 20)
discretise_data['fico_range_high'] = pd.cut(discretise_data['fico_range_high'], bins= 20)

est = KBinsDiscretizer(n_bins= 6, encode='ordinal', strategy='kmeans')
inq_last_6mths_dataframe = pd.DataFrame(discretise_data['inq_last_6mths'])
est.fit(inq_last_6mths_dataframe)
inq_last_6mths_dataframe = est.transform(inq_last_6mths_dataframe)
discretise_data['inq_last_6mths'] = inq_last_6mths_dataframe


est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
open_acc_dataframe = pd.DataFrame(discretise_data['open_acc'])
est.fit(open_acc_dataframe)
open_acc_dataframe = est.transform(open_acc_dataframe)
discretise_data['open_acc'] = open_acc_dataframe

est = KBinsDiscretizer(n_bins= 34, encode='ordinal', strategy='kmeans')
pub_rec_dataframe = pd.DataFrame(discretise_data['pub_rec'])
est.fit(pub_rec_dataframe)
pub_rec_dataframe = est.transform(pub_rec_dataframe)
discretise_data['pub_rec'] = pub_rec_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
revol_bal_dataframe = pd.DataFrame(discretise_data['revol_bal'])
est.fit(revol_bal_dataframe)
revol_bal_dataframe = est.transform(revol_bal_dataframe)
discretise_data['revol_bal'] = revol_bal_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
revol_util_dataframe = pd.DataFrame(discretise_data['revol_util'])
est.fit(revol_util_dataframe)
revol_util_dataframe = est.transform(revol_util_dataframe)
discretise_data['revol_util'] = revol_util_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
total_acc_dataframe = pd.DataFrame(discretise_data['total_acc'])
est.fit(total_acc_dataframe)
total_acc_dataframe = est.transform(total_acc_dataframe)
discretise_data['total_acc'] = total_acc_dataframe



est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
out_prncp_dataframe = pd.DataFrame(discretise_data['out_prncp'])
est.fit(out_prncp_dataframe)
out_prncp_dataframe = est.transform(out_prncp_dataframe)
discretise_data['out_prncp'] = out_prncp_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
out_prncp_inv_dataframe = pd.DataFrame(discretise_data['out_prncp_inv'])
est.fit(out_prncp_inv_dataframe)
out_prncp_inv_dataframe = est.transform(out_prncp_inv_dataframe)
discretise_data['out_prncp_inv'] = out_prncp_inv_dataframe


#discretise_data['total_pymnt'] = pd.cut(discretise_data['total_pymnt'], bins= 50)
est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
total_pymnt_dataframe = pd.DataFrame(discretise_data['total_pymnt'])
est.fit(total_pymnt_dataframe)
total_pymnt_dataframe = est.transform(total_pymnt_dataframe)
discretise_data['total_pymnt'] = total_pymnt_dataframe

#discretise_data['total_pymnt_inv'] = pd.cut(discretise_data['total_pymnt_inv'], bins= 50)
est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
total_pymnt_inv_dataframe = pd.DataFrame(discretise_data['total_pymnt_inv'])
est.fit(total_pymnt_inv_dataframe)
total_pymnt_inv_dataframe = est.transform(total_pymnt_inv_dataframe)
discretise_data['total_pymnt_inv'] = total_pymnt_inv_dataframe

#discretise_data['total_rec_prncp'] = pd.cut(discretise_data['total_rec_prncp'], bins= 50)
est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
total_rec_prncp_dataframe = pd.DataFrame(discretise_data['total_rec_prncp'])
est.fit(total_rec_prncp_dataframe)
total_rec_prncp_dataframe = est.transform(total_rec_prncp_dataframe)
discretise_data['total_rec_prncp'] = total_rec_prncp_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
total_rec_int_dataframe = pd.DataFrame(discretise_data['total_rec_int'])
est.fit(total_rec_int_dataframe)
total_rec_int_dataframe = est.transform(total_rec_int_dataframe)
discretise_data['total_rec_int'] = total_rec_int_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
total_rec_late_fee_dataframe = pd.DataFrame(discretise_data['total_rec_late_fee'])
est.fit(total_rec_late_fee_dataframe)
total_rec_late_fee_dataframe = est.transform(total_rec_late_fee_dataframe)
discretise_data['total_rec_late_fee'] = total_rec_late_fee_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
recoveries_dataframe = pd.DataFrame(discretise_data['recoveries'])
est.fit(recoveries_dataframe)
recoveries_dataframe = est.transform(recoveries_dataframe)
discretise_data['recoveries'] = recoveries_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
collection_recovery_fee_dataframe = pd.DataFrame(discretise_data['collection_recovery_fee'])
est.fit(collection_recovery_fee_dataframe)
collection_recovery_fee_dataframe = est.transform(collection_recovery_fee_dataframe)
discretise_data['collection_recovery_fee'] = collection_recovery_fee_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
last_pymnt_amnt_dataframe = pd.DataFrame(discretise_data['last_pymnt_amnt'])
est.fit(last_pymnt_amnt_dataframe)
last_pymnt_amnt_dataframe = est.transform(last_pymnt_amnt_dataframe)
discretise_data['last_pymnt_amnt'] = last_pymnt_amnt_dataframe

discretise_data['last_fico_range_high'] = pd.cut(discretise_data['last_fico_range_high'], bins= 50)
discretise_data['last_fico_range_low'] = pd.cut(discretise_data['last_fico_range_low'], bins= 50)

discretise_data['collections_12_mths_ex_med'] = pd.cut(discretise_data['collections_12_mths_ex_med'], bins= 10)

discretise_data['policy_code'] = pd.cut(discretise_data['policy_code'], bins= 2, labels= [1,0])

discretise_data['acc_now_delinq'] = pd.cut(discretise_data['acc_now_delinq'], bins= 7)

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
tot_coll_amt_dataframe = pd.DataFrame(discretise_data['tot_coll_amt'])
est.fit(tot_coll_amt_dataframe)
tot_coll_amt_dataframe = est.transform(tot_coll_amt_dataframe)
discretise_data['tot_coll_amt'] = tot_coll_amt_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
tot_cur_bal_dataframe = pd.DataFrame(discretise_data['tot_cur_bal'])
est.fit(tot_cur_bal_dataframe)
tot_cur_bal_dataframe = est.transform(tot_cur_bal_dataframe)
discretise_data['tot_cur_bal'] = tot_cur_bal_dataframe

est = KBinsDiscretizer(n_bins= 17, encode='ordinal', strategy='kmeans')
open_acc_6m_dataframe = pd.DataFrame(discretise_data['open_acc_6m'])
est.fit(open_acc_6m_dataframe)
open_acc_6m_dataframe = est.transform(open_acc_6m_dataframe)
discretise_data['open_acc_6m'] = open_acc_6m_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
open_act_il_dataframe = pd.DataFrame(discretise_data['open_act_il'])
est.fit(open_act_il_dataframe)
open_act_il_dataframe = est.transform(open_act_il_dataframe)
discretise_data['open_act_il'] = open_act_il_dataframe

est = KBinsDiscretizer(n_bins= 15, encode='ordinal', strategy='kmeans')
open_il_12m_dataframe = pd.DataFrame(discretise_data['open_il_12m'])
est.fit(open_il_12m_dataframe)
open_il_12m_dataframe = est.transform(open_il_12m_dataframe)
discretise_data['open_il_12m'] = open_il_12m_dataframe

est = KBinsDiscretizer(n_bins= 27, encode='ordinal', strategy='kmeans')
open_il_24m_dataframe = pd.DataFrame(discretise_data['open_il_24m'])
est.fit(open_il_24m_dataframe)
open_il_24m_dataframe = est.transform(open_il_24m_dataframe)
discretise_data['open_il_24m'] = open_il_24m_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
total_bal_il_dataframe = pd.DataFrame(discretise_data['total_bal_il'])
est.fit(total_bal_il_dataframe)
total_bal_il_dataframe = est.transform(total_bal_il_dataframe)
discretise_data['total_bal_il'] = total_bal_il_dataframe

est = KBinsDiscretizer(n_bins= 28, encode='ordinal', strategy='kmeans')
open_rv_12m_dataframe = pd.DataFrame(discretise_data['open_rv_12m'])
est.fit(open_rv_12m_dataframe)
open_rv_12m_dataframe = est.transform(open_rv_12m_dataframe)
discretise_data['open_rv_12m'] = open_rv_12m_dataframe


est = KBinsDiscretizer(n_bins= 50 , encode='ordinal', strategy='kmeans')
open_rv_24m_dataframe = pd.DataFrame(discretise_data['open_rv_24m'])
est.fit(open_rv_24m_dataframe)
open_rv_24m_dataframe = est.transform(open_rv_24m_dataframe)
discretise_data['open_rv_24m'] = open_rv_24m_dataframe


est = KBinsDiscretizer(n_bins= 50 , encode='ordinal', strategy='kmeans')
max_bal_bc_dataframe = pd.DataFrame(discretise_data['max_bal_bc'])
est.fit(max_bal_bc_dataframe)
max_bal_bc_dataframe = est.transform(max_bal_bc_dataframe)
discretise_data['max_bal_bc'] = max_bal_bc_dataframe


est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
all_util_dataframe = pd.DataFrame(discretise_data['all_util'])
est.fit(all_util_dataframe)
all_util_dataframe = est.transform(all_util_dataframe)
discretise_data['all_util'] = all_util_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
total_rev_hi_lim_dataframe = pd.DataFrame(discretise_data['total_rev_hi_lim'])
est.fit(total_rev_hi_lim_dataframe)
total_rev_hi_lim_dataframe = est.transform(total_rev_hi_lim_dataframe)
discretise_data['total_rev_hi_lim'] = total_rev_hi_lim_dataframe

est = KBinsDiscretizer(n_bins= 33, encode='ordinal', strategy='kmeans')
inq_fi_dataframe = pd.DataFrame(discretise_data['inq_fi'])
est.fit(inq_fi_dataframe)
inq_fi_dataframe = est.transform(inq_fi_dataframe)
discretise_data['inq_fi'] = inq_fi_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
total_cu_tl_dataframe = pd.DataFrame(discretise_data['total_cu_tl'])
est.fit(total_cu_tl_dataframe)
total_cu_tl_dataframe = est.transform(total_cu_tl_dataframe)
discretise_data['total_cu_tl'] = total_cu_tl_dataframe

est = KBinsDiscretizer(n_bins= 48, encode='ordinal', strategy='kmeans')
inq_last_12m_dataframe = pd.DataFrame(discretise_data['inq_last_12m'])
est.fit(inq_last_12m_dataframe)
inq_last_12m_dataframe = est.transform(inq_last_12m_dataframe)
discretise_data['inq_last_12m'] = inq_last_12m_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
acc_open_past_24mths_dataframe = pd.DataFrame(discretise_data['acc_open_past_24mths'])
est.fit(acc_open_past_24mths_dataframe)
acc_open_past_24mths_dataframe = est.transform(acc_open_past_24mths_dataframe)
discretise_data['acc_open_past_24mths'] = acc_open_past_24mths_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
avg_cur_bal_dataframe = pd.DataFrame(discretise_data['avg_cur_bal'])
est.fit(avg_cur_bal_dataframe)
avg_cur_bal_dataframe = est.transform(avg_cur_bal_dataframe)
discretise_data['avg_cur_bal'] = avg_cur_bal_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
bc_open_to_buy_dataframe = pd.DataFrame(discretise_data['bc_open_to_buy'])
est.fit(bc_open_to_buy_dataframe)
bc_open_to_buy_dataframe = est.transform(bc_open_to_buy_dataframe)
discretise_data['bc_open_to_buy'] = bc_open_to_buy_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
bc_util_dataframe = pd.DataFrame(discretise_data['bc_util'])
est.fit(bc_util_dataframe)
bc_util_dataframe = est.transform(bc_util_dataframe)
discretise_data['bc_util'] = bc_util_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
delinq_amnt_dataframe = pd.DataFrame(discretise_data['delinq_amnt'])
est.fit(delinq_amnt_dataframe)
delinq_amnt_dataframe = est.transform(delinq_amnt_dataframe)
discretise_data['delinq_amnt'] = delinq_amnt_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
mo_sin_old_il_acct_dataframe = pd.DataFrame(discretise_data['mo_sin_old_il_acct'])
est.fit(mo_sin_old_il_acct_dataframe)
mo_sin_old_il_acct_dataframe = est.transform(mo_sin_old_il_acct_dataframe)
discretise_data['mo_sin_old_il_acct'] = mo_sin_old_il_acct_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
mo_sin_old_rev_tl_op_dataframe = pd.DataFrame(discretise_data['mo_sin_old_rev_tl_op'])
est.fit(mo_sin_old_rev_tl_op_dataframe)
mo_sin_old_rev_tl_op_dataframe = est.transform(mo_sin_old_rev_tl_op_dataframe)
discretise_data['mo_sin_old_rev_tl_op'] = mo_sin_old_rev_tl_op_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
mo_sin_rcnt_rev_tl_op_dataframe = pd.DataFrame(discretise_data['mo_sin_rcnt_rev_tl_op'])
est.fit(mo_sin_rcnt_rev_tl_op_dataframe)
mo_sin_rcnt_rev_tl_op_dataframe = est.transform(mo_sin_rcnt_rev_tl_op_dataframe)
discretise_data['mo_sin_rcnt_rev_tl_op'] = mo_sin_rcnt_rev_tl_op_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
mo_sin_rcnt_tl_dataframe = pd.DataFrame(discretise_data['mo_sin_rcnt_tl'])
est.fit(mo_sin_rcnt_tl_dataframe)
mo_sin_rcnt_tl_dataframe = est.transform(mo_sin_rcnt_tl_dataframe)
discretise_data['mo_sin_rcnt_tl'] = mo_sin_rcnt_tl_dataframe

est = KBinsDiscretizer(n_bins= 41, encode='ordinal', strategy='kmeans')
mort_acc_dataframe = pd.DataFrame(discretise_data['mort_acc'])
est.fit(mort_acc_dataframe)
mort_acc_dataframe = est.transform(mort_acc_dataframe)
discretise_data['mort_acc'] = mort_acc_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
mths_since_recent_bc_dataframe = pd.DataFrame(discretise_data['mths_since_recent_bc'])
est.fit(mths_since_recent_bc_dataframe)
mths_since_recent_bc_dataframe = est.transform(mths_since_recent_bc_dataframe)
discretise_data['mths_since_recent_bc'] = mths_since_recent_bc_dataframe

est = KBinsDiscretizer(n_bins= 25, encode='ordinal', strategy='kmeans')
mths_since_recent_inq_dataframe = pd.DataFrame(discretise_data['mths_since_recent_inq'])
est.fit(mths_since_recent_inq_dataframe)
mths_since_recent_inq_dataframe = est.transform(mths_since_recent_inq_dataframe)
discretise_data['mths_since_recent_inq'] = mths_since_recent_inq_dataframe

est = KBinsDiscretizer(n_bins= 41, encode='ordinal', strategy='kmeans')
num_accts_ever_120_pd_dataframe = pd.DataFrame(discretise_data['num_accts_ever_120_pd'])
est.fit(num_accts_ever_120_pd_dataframe)
num_accts_ever_120_pd_dataframe = est.transform(num_accts_ever_120_pd_dataframe)
discretise_data['num_accts_ever_120_pd'] = num_accts_ever_120_pd_dataframe

est = KBinsDiscretizer(n_bins= 36, encode='ordinal', strategy='kmeans')
num_actv_bc_tl_dataframe = pd.DataFrame(discretise_data['num_actv_bc_tl'])
est.fit(num_actv_bc_tl_dataframe)
num_actv_bc_tl_dataframe = est.transform(num_actv_bc_tl_dataframe)
discretise_data['num_actv_bc_tl'] = num_actv_bc_tl_dataframe

est = KBinsDiscretizer(n_bins= 49, encode='ordinal', strategy='kmeans')
num_actv_rev_tl_dataframe = pd.DataFrame(discretise_data['num_actv_rev_tl'])
est.fit(num_actv_rev_tl_dataframe)
num_actv_rev_tl_dataframe = est.transform(num_actv_rev_tl_dataframe)
discretise_data['num_actv_rev_tl'] = num_actv_rev_tl_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
num_bc_sats_dataframe = pd.DataFrame(discretise_data['num_bc_sats'])
est.fit(num_bc_sats_dataframe)
num_bc_sats_dataframe = est.transform(num_bc_sats_dataframe)
discretise_data['num_bc_sats'] = num_bc_sats_dataframe


est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
num_bc_tl_dataframe = pd.DataFrame(discretise_data['num_bc_tl'])
est.fit(num_bc_tl_dataframe)
num_bc_tl_dataframe = est.transform(num_bc_tl_dataframe)
discretise_data['num_bc_tl'] = num_bc_tl_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
num_il_tl_dataframe = pd.DataFrame(discretise_data['num_il_tl'])
est.fit(num_il_tl_dataframe)
num_il_tl_dataframe = est.transform(num_il_tl_dataframe)
discretise_data['num_il_tl'] = num_il_tl_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
num_op_rev_tl_dataframe = pd.DataFrame(discretise_data['num_op_rev_tl'])
est.fit(num_op_rev_tl_dataframe)
num_op_rev_tl_dataframe = est.transform(num_op_rev_tl_dataframe)
discretise_data['num_op_rev_tl'] = num_op_rev_tl_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
num_rev_accts_dataframe = pd.DataFrame(discretise_data['num_rev_accts'])
est.fit(num_rev_accts_dataframe)
num_rev_accts_dataframe = est.transform(num_rev_accts_dataframe)
discretise_data['num_rev_accts'] = num_rev_accts_dataframe

est = KBinsDiscretizer(n_bins= 49, encode='ordinal', strategy='kmeans')
num_rev_tl_bal_gt_0_dataframe = pd.DataFrame(discretise_data['num_rev_tl_bal_gt_0'])
est.fit(num_rev_tl_bal_gt_0_dataframe)
num_rev_tl_bal_gt_0_dataframe = est.transform(num_rev_tl_bal_gt_0_dataframe)
discretise_data['num_rev_tl_bal_gt_0'] = num_rev_tl_bal_gt_0_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
num_sats_dataframe = pd.DataFrame(discretise_data['num_sats'])
est.fit(num_sats_dataframe)
num_sats_dataframe = est.transform(num_sats_dataframe)
discretise_data['num_sats'] = num_sats_dataframe

est = KBinsDiscretizer(n_bins= 6, encode='ordinal', strategy='kmeans')
num_tl_120dpd_2m_dataframe = pd.DataFrame(discretise_data['num_tl_120dpd_2m'])
est.fit(num_tl_120dpd_2m_dataframe)
num_tl_120dpd_2m_dataframe = est.transform(num_tl_120dpd_2m_dataframe)
discretise_data['num_tl_120dpd_2m'] = num_tl_120dpd_2m_dataframe

est = KBinsDiscretizer(n_bins= 5, encode='ordinal', strategy='kmeans')
num_tl_30dpd_dataframe = pd.DataFrame(discretise_data['num_tl_30dpd'])
est.fit(num_tl_30dpd_dataframe)
num_tl_30dpd_dataframe = est.transform(num_tl_30dpd_dataframe)
discretise_data['num_tl_30dpd'] = num_tl_30dpd_dataframe

est = KBinsDiscretizer(n_bins= 29, encode='ordinal', strategy='kmeans')
num_tl_90g_dpd_24m_dataframe = pd.DataFrame(discretise_data['num_tl_90g_dpd_24m'])
est.fit(num_tl_90g_dpd_24m_dataframe)
num_tl_90g_dpd_24m_dataframe = est.transform(num_tl_90g_dpd_24m_dataframe)
discretise_data['num_tl_90g_dpd_24m'] = num_tl_90g_dpd_24m_dataframe

est = KBinsDiscretizer(n_bins= 30, encode='ordinal', strategy='kmeans')
num_tl_op_past_12m_dataframe = pd.DataFrame(discretise_data['num_tl_op_past_12m'])
est.fit(num_tl_op_past_12m_dataframe)
num_tl_op_past_12m_dataframe = est.transform(num_tl_op_past_12m_dataframe)
discretise_data['num_tl_op_past_12m'] = num_tl_op_past_12m_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
pct_tl_nvr_dlq_dataframe = pd.DataFrame(discretise_data['pct_tl_nvr_dlq'])
est.fit(pct_tl_nvr_dlq_dataframe)
pct_tl_nvr_dlq_dataframe = est.transform(pct_tl_nvr_dlq_dataframe)
discretise_data['pct_tl_nvr_dlq'] = pct_tl_nvr_dlq_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
percent_bc_gt_75_dataframe = pd.DataFrame(discretise_data['percent_bc_gt_75'])
est.fit(percent_bc_gt_75_dataframe)
percent_bc_gt_75_dataframe = est.transform(percent_bc_gt_75_dataframe)
discretise_data['percent_bc_gt_75'] = percent_bc_gt_75_dataframe

est = KBinsDiscretizer(n_bins= 10, encode='ordinal', strategy='kmeans')
pub_rec_bankruptcies_dataframe = pd.DataFrame(discretise_data['pub_rec_bankruptcies'])
est.fit(pub_rec_bankruptcies_dataframe)
pub_rec_bankruptcies_dataframe = est.transform(pub_rec_bankruptcies_dataframe)
discretise_data['pub_rec_bankruptcies'] = pub_rec_bankruptcies_dataframe

est = KBinsDiscretizer(n_bins= 32, encode='ordinal', strategy='kmeans')
tax_liens_dataframe = pd.DataFrame(discretise_data['tax_liens'])
est.fit(tax_liens_dataframe)
tax_liens_dataframe = est.transform(tax_liens_dataframe)
discretise_data['tax_liens'] = tax_liens_dataframe


est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
tot_hi_cred_lim_dataframe = pd.DataFrame(discretise_data['tot_hi_cred_lim'])
est.fit(tot_hi_cred_lim_dataframe)
tot_hi_cred_lim_dataframe = est.transform(tot_hi_cred_lim_dataframe)
discretise_data['tot_hi_cred_lim'] = tot_hi_cred_lim_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
total_bal_ex_mort_dataframe = pd.DataFrame(discretise_data['total_bal_ex_mort'])
est.fit(total_bal_ex_mort_dataframe)
total_bal_ex_mort_dataframe = est.transform(total_bal_ex_mort_dataframe)
discretise_data['total_bal_ex_mort'] = total_bal_ex_mort_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
total_bc_limit_dataframe = pd.DataFrame(discretise_data['total_bc_limit'])
est.fit(total_bc_limit_dataframe)
total_bc_limit_dataframe = est.transform(total_bc_limit_dataframe)
discretise_data['total_bc_limit'] = total_bc_limit_dataframe

est = KBinsDiscretizer(n_bins= 50, encode='ordinal', strategy='kmeans')
total_il_high_credit_limit_dataframe = pd.DataFrame(discretise_data['total_il_high_credit_limit'])
est.fit(total_il_high_credit_limit_dataframe)
total_il_high_credit_limit_dataframe = est.transform(total_il_high_credit_limit_dataframe)
discretise_data['total_il_high_credit_limit'] = total_il_high_credit_limit_dataframe



display(discretise_data)


c:\Users\zachs\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
c:\Users\zachs\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(
c:\Users\zachs\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,"(3340.0, 4120.0]","(3340.0, 4120.0]","(3081.5, 3867.0]",36 months,"(13.528, 14.041]","(110.343, 144.588]",C,C4,leadman,10+ years,...,0.0,0.0,0.0,5.0,0.0,0.0,1.0,N,Cash,N
1,"(24400.0, 25180.0]","(24400.0, 25180.0]","(24290.0, 25075.5]",36 months,"(11.987, 12.5]","(795.231, 829.476]",C,C1,Engineer,10+ years,...,3.0,0.0,0.0,9.0,3.0,13.0,2.0,N,Cash,N
4,"(10360.0, 11140.0]","(10360.0, 11140.0]","(10151.0, 10936.5]",60 months,"(22.259, 22.772]","(281.565, 315.81]",F,F1,Contract Specialist,3 years,...,30.0,0.0,0.0,12.0,9.0,4.0,8.0,N,Cash,N
6,"(19720.0, 20500.0]","(19720.0, 20500.0]","(19577.0, 20362.5]",36 months,"(8.905, 9.419]","(624.009, 658.254]",B,B2,Vice President of Recruiting Operations,10+ years,...,49.0,0.0,0.0,11.0,11.0,6.0,5.0,N,Cash,N
7,"(19720.0, 20500.0]","(19720.0, 20500.0]","(19577.0, 20362.5]",36 months,"(8.392, 8.905]","(624.009, 658.254]",B,B1,road driver,10+ years,...,0.0,0.0,0.0,6.0,2.0,2.0,4.0,N,Cash,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260691,"(31420.0, 32200.0]","(31420.0, 32200.0]","(31359.5, 32145.0]",60 months,"(14.041, 14.555]","(726.742, 760.987]",C,C4,Sales Manager,3 years,...,0.0,0.0,0.0,14.0,12.0,3.0,2.0,N,Cash,N
2260692,"(15820.0, 16600.0]","(15820.0, 16600.0]","(15649.5, 16435.0]",60 months,"(12.5, 13.014]","(350.054, 384.298]",C,C1,Manager,10+ years,...,26.0,3.0,0.0,2.0,6.0,1.0,7.0,N,Cash,N
2260693,"(23620.0, 24400.0]","(23620.0, 24400.0]","(23504.5, 24290.0]",60 months,"(10.446, 10.96]","(487.032, 521.276]",B,B3,Current Operations Officer,4 years,...,0.0,0.0,2.0,4.0,3.0,15.0,3.0,N,Cash,N
2260695,"(23620.0, 24400.0]","(23620.0, 24400.0]","(23504.5, 24290.0]",60 months,"(10.446, 10.96]","(487.032, 521.276]",B,B3,Database Administrator,10+ years,...,22.0,0.0,1.0,6.0,14.0,5.0,14.0,N,Cash,N
